In [ ]:
from theano.sandbox import cuda

In [2]:
#set FIXED variables
HOMEPATH = "/home/ubuntu/fastai/"


import os, errno

from datetime import datetime
from shutil import copyfile, move
import random

In [3]:
os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())

%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

('current working directory:', '/home/ubuntu/fastai')


Using TensorFlow backend.


In [4]:
os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())

#path = "data/ml-20m/" # large dataset http://files.grouplens.org/datasets/movielens/ml-20m.zip
path = "data/ml-small/" #small dataset http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
print ("path:", os.getcwd()+path)
model_path = path + 'models/'
if not os.path.exists(model_path): 
    os.mkdir(model_path)

    

batch_size=64

current working directory: /home/ubuntu/fastai
path: /home/ubuntu/fastaidata/ml-small/


## Set up data

We're working with the movielens data, which contains one rating per row, like this:

userId,movieId,rating,timestamp  
1,31,2.5,1260759144  
1,1029,3.0,1260759179  
1,1061,3.0,1260759182  

In [20]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [21]:
print ("ratings:", type(ratings), len(ratings), ratings.shape)

ratings: <class 'pandas.core.frame.DataFrame'> 100004 (100004, 4)


Just for display purposes, let's read in the movie names too.

movieId,title,genres  
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy  
2,Jumanji (1995),Adventure|Children|Fantasy  
3,Grumpier Old Men (1995),Comedy|Romance  


In [22]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [23]:
print (type(movie_names), len(movie_names))

<type 'dict'> 9125


In [24]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()
print ("users:", type(users), users.shape)
print ("movies:", type(movies), movies.shape)
print (users[0:5])
print (movies[0:5])

users: <type 'numpy.ndarray'> (671,)
movies: <type 'numpy.ndarray'> (9066,)
[1 2 3 4 5]
[  31 1029 1061 1129 1172]


In [25]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}
print (type(userid2idx), len(userid2idx))
print(type(movieid2idx), len(movieid2idx))


from itertools import islice
print ("userid2idx:",  list(islice(userid2idx.iteritems(), 5)))
print ("movieid2idx:",  list(islice(movieid2idx.iteritems(), 5)))

<type 'dict'> 671
<type 'dict'> 9066
userid2idx: [(1, 0), (2, 1), (3, 2), (4, 3), (5, 4)]
movieid2idx: [(1, 417), (2, 650), (3, 319), (4, 2084), (5, 651)]


We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.

In [26]:
#show values before
print ("ratings.movieId\n", ratings.movieId[0:5])
print ("ratings.userId\n", ratings.userId[0:5])

ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

#show values after
print ("ratings.movieId\n", ratings.movieId[0:5])
print ("ratings.userId\n", ratings.userId[0:5])


ratings.movieId
 0      31
1    1029
2    1061
3    1129
4    1172
Name: movieId, dtype: int64
ratings.userId
 0    1
1    1
2    1
3    1
4    1
Name: userId, dtype: int64
ratings.movieId
 0    0
1    1
2    2
3    3
4    4
Name: movieId, dtype: int64
ratings.userId
 0    0
1    0
2    0
3    0
4    0
Name: userId, dtype: int64


In [27]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
                                            ratings.userId.max(), 
                                            ratings.movieId.min(), 
                                            ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [28]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

This is the number of latent factors in each embedding.

In [29]:
n_factors = 50

In [30]:
np.random.seed = 42

Randomly split into training and validation.

In [38]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]
#~ operator inverts selection 
#trn & val are non overlapping and complimentry subsets of ratings
print ("ratings:", type(ratings), ratings.shape)
print ("msk:", type(msk), msk.shape)
print ("trn:", type(trn), trn.shape)
print ("val:", type(val), val.shape)

ratings: <class 'pandas.core.frame.DataFrame'> (100004, 4)
msk: <type 'numpy.ndarray'> (100004,)
trn: <class 'pandas.core.frame.DataFrame'> (80074, 4)
val: <class 'pandas.core.frame.DataFrame'> (19930, 4)


## Create subset for Excel

We create a crosstab of the most popular movies and most movie-addicted users which we'll copy into Excel for creating a simple example. This isn't necessary for any of the modeling below however.

In [32]:
g=ratings.groupby('userId')['rating'].count()
topUsers=g.sort_values(ascending=False)[:15]

In [33]:
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:15]

In [34]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')

In [35]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieId')

In [36]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


## Dot product

The most basic model is a dot product of a movie embedding and a user embedding. Let's see how well that works:  

https://keras.io/layers/embeddings/
Turns positive integers (indexes) into dense vectors of fixed size. 
This layer can only be used as the first layer in a model.

In [39]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

print ("user_in:", type(user_in))
print ("u:", type(u))
print ("movie_in:", type(movie_in))
print ("m:", type(m))

user_in: <class 'tensorflow.python.framework.ops.Tensor'>
u: <class 'tensorflow.python.framework.ops.Tensor'>
movie_in: <class 'tensorflow.python.framework.ops.Tensor'>
m: <class 'tensorflow.python.framework.ops.Tensor'>


In [42]:
#https://keras.io/models/model/
#

x = merge([u, m], mode='dot')
print ("x:", type(x))
x = Flatten()(x)
model = Model([user_in, movie_in], x)
print ("model:", type(model))
model.compile(Adam(0.001), loss='mse')

x: <class 'tensorflow.python.framework.ops.Tensor'>
model: <class 'keras.engine.training.Model'>


In [44]:
startTime= datetime.now()

model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val.userId, val.movieId], val.rating))
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


Train on 80074 samples, validate on 19930 samples
Epoch 1/1
80074/80074 [==============================] - 311s - loss: 9.9531 - val_loss: 4.3444
Time elpased (hh:mm:ss.ms) 0:05:11.342598


In [45]:
model.optimizer.lr=0.01

In [46]:
startTime= datetime.now()
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=3, 
          validation_data=([val.userId, val.movieId], val.rating))
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


Train on 80074 samples, validate on 19930 samples
Epoch 1/3
80074/80074 [==============================] - 432s - loss: 3.1344 - val_loss: 2.8769
Epoch 2/3
80074/80074 [==============================] - 365s - loss: 2.4137 - val_loss: 2.6692
Epoch 3/3
80074/80074 [==============================] - 362s - loss: 2.2529 - val_loss: 2.6241
Time elpased (hh:mm:ss.ms) 0:19:20.793292


In [ ]:
model.optimizer.lr=0.001

In [47]:
startTime= datetime.now()
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


Train on 80074 samples, validate on 19930 samples
Epoch 1/6
80074/80074 [==============================] - 363s - loss: 2.1941 - val_loss: 2.6036
Epoch 2/6
80074/80074 [==============================] - 362s - loss: 2.1603 - val_loss: 2.6026
Epoch 3/6
80074/80074 [==============================] - 362s - loss: 2.1333 - val_loss: 2.5993
Epoch 4/6
80074/80074 [==============================] - 361s - loss: 2.1052 - val_loss: 2.6014
Epoch 5/6
80074/80074 [==============================] - 362s - loss: 2.0800 - val_loss: 2.6065
Epoch 6/6
80074/80074 [==============================] - 362s - loss: 2.0567 - val_loss: 2.6129
Time elpased (hh:mm:ss.ms) 0:36:15.940990


The [best benchmarks](http://www.librec.net/example.html) are a bit over 0.9, so this model doesn't seem to be working that well...

##  Bias

The problem is likely to be that we don't have bias terms - that is, a single bias for each user and each movie representing how positive or negative each user is, and how good each movie is. We can add that easily by simply creating an embedding with one output for each movie and each user, and adding it to our output.  

lesson 5 video @ 14:35

https://keras.io/regularizers/  
Regularizers allow to apply penalties on layer parameters or layer activity during optimization.   
These penalties are incorporated in the loss function that the network optimizes.  


In [48]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

#l2 norm = sum of squares

In [49]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [50]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [51]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [52]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [53]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80074 samples, validate on 19930 samples
Epoch 1/1
80074/80074 [==============================] - 420s - loss: 8.7905 - val_loss: 3.5505


In [54]:
model.optimizer.lr=0.01

In [55]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80074 samples, validate on 19930 samples
Epoch 1/6
80074/80074 [==============================] - 523s - loss: 2.5835 - val_loss: 2.3516
Epoch 2/6
80074/80074 [==============================] - 504s - loss: 1.9950 - val_loss: 2.1390
Epoch 3/6
80074/80074 [==============================] - 363s - loss: 1.8320 - val_loss: 2.0371
Epoch 4/6
80074/80074 [==============================] - 444s - loss: 1.7363 - val_loss: 1.9583
Epoch 5/6
80074/80074 [==============================] - 511s - loss: 1.6551 - val_loss: 1.8868
Epoch 6/6
80074/80074 [==============================] - 492s - loss: 1.5788 - val_loss: 1.8196


In [56]:
model.optimizer.lr=0.001

In [57]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80074 samples, validate on 19930 samples
Epoch 1/10
80074/80074 [==============================] - 363s - loss: 1.5080 - val_loss: 1.7551
Epoch 2/10
80074/80074 [==============================] - 441s - loss: 1.4377 - val_loss: 1.6957
Epoch 3/10
80074/80074 [==============================] - 512s - loss: 1.3707 - val_loss: 1.6401
Epoch 4/10
80074/80074 [==============================] - 493s - loss: 1.3057 - val_loss: 1.5812
Epoch 5/10
80074/80074 [==============================] - 363s - loss: 1.2423 - val_loss: 1.5302
Epoch 6/10
80074/80074 [==============================] - 441s - loss: 1.1829 - val_loss: 1.4793
Epoch 7/10
80074/80074 [==============================] - 515s - loss: 1.1264 - val_loss: 1.4318
Epoch 8/10
80074/80074 [==============================] - 492s - loss: 1.0721 - val_loss: 1.3825
Epoch 9/10
80074/80074 [==============================] - 363s - loss: 1.0194 - val_loss: 1.3418
Epoch 10/10
80074/80074 [==============================] - 404s - loss: 0.970

In [58]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80074 samples, validate on 19930 samples
Epoch 1/5
80074/80074 [==============================] - 524s - loss: 0.9242 - val_loss: 1.2671
Epoch 2/5
80074/80074 [==============================] - 516s - loss: 0.8803 - val_loss: 1.2371
Epoch 3/5
80074/80074 [==============================] - 78s - loss: 0.8399 - val_loss: 1.2044
Epoch 4/5
80074/80074 [==============================] - 6s - loss: 0.8024 - val_loss: 1.1774
Epoch 5/5
80074/80074 [==============================] - 6s - loss: 0.7680 - val_loss: 1.1527


This result is quite a bit better than the best benchmarks that we could find with a quick google search - so looks like a great approach!

In [59]:
model.save_weights(model_path+'bias.h5')

In [60]:
model.load_weights(model_path+'bias.h5')

We can use the model to generate predictions by passing a pair of ints - a user id and a movie id. For instance, this predicts that user #3 would really enjoy movie #6.

In [61]:
model.predict([np.array([3]), np.array([6])])

array([[ 4.8289]], dtype=float32)

## Analyze results

To make the analysis of the factors more interesting, we'll restrict it to the top 2000 most popular movies.

In [62]:
g=ratings.groupby('movieId')['rating'].count()
print (type(g))
topMovies=g.sort_values(ascending=False)[:2000]
print ("topMovies:", type(topMovies))
topMovies = np.array(topMovies.index)

<class 'pandas.core.series.Series'>
topMovies: <class 'pandas.core.series.Series'>


First, we'll look at the movie bias term. We create a 'model' - which in keras is simply a way of associating one or more inputs with one more more outputs, using the functional API. Here, our input is the movie id (a single id), and the output is the movie bias (a single float).

In [63]:
get_movie_bias = Model(movie_in, mb)
print ("get_movie_bias:", type(get_movie_bias))
movie_bias = get_movie_bias.predict(topMovies)
print ("movie_bias:", type(movie_bias))
movie_ratings = [(b[0], movie_names[movies[i]]) for i,b in zip(topMovies,movie_bias)]
print ("movie_ratings:", type(movie_ratings))

get_movie_bias: <class 'keras.engine.training.Model'>
movie_bias: <type 'numpy.ndarray'>
movie_ratings: <type 'list'>


Now we can look at the top and bottom rated movies. These ratings are corrected for different levels of reviewer sentiment, as well as different types of movies that different reviewers watch.

In [ ]:
sorted(movie_ratings, key=itemgetter(0))[:15]


In [ ]:
sorted(movie_ratings, key=itemgetter(0), reverse=True)[:15]

We can now do the same thing for the embeddings.

In [ ]:
get_movie_emb = Model(movie_in, m)
movie_emb = np.squeeze(get_movie_emb.predict([topMovies]))
movie_emb.shape

Because it's hard to interpret 50 embeddings, we use [PCA](https://plot.ly/ipython-notebooks/principal-component-analysis/) to simplify them down to just 3 vectors. 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
movie_pca = pca.fit(movie_emb.T).components_

In [ ]:
fac0 = movie_pca[0]

In [ ]:
movie_comp = [(f, movie_names[movies[i]]) for f,i in zip(fac0, topMovies)]

Here's the 1st component. It seems to be 'critically acclaimed' or 'classic'.

In [ ]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp, key=itemgetter(0))[:10]

In [ ]:
fac1 = movie_pca[1]

In [ ]:
movie_comp = [(f, movie_names[movies[i]]) for f,i in zip(fac1, topMovies)]

The 2nd is 'hollywood blockbuster'.

In [ ]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp, key=itemgetter(0))[:10]

In [ ]:
fac2 = movie_pca[2]

In [ ]:
movie_comp = [(f, movie_names[movies[i]]) for f,i in zip(fac2, topMovies)]

The 3rd is 'violent vs happy'.

In [ ]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]

In [ ]:
sorted(movie_comp, key=itemgetter(0))[:10]

We can draw a picture to see how various movies appear on the map of these components. This picture shows the 1st and 3rd components.

In [ ]:
import sys
stdout, stderr = sys.stdout, sys.stderr # save notebook stdout and stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout, sys.stderr = stdout, stderr # restore notebook stdout and stderr

In [ ]:
start=50; end=100
X = fac0[start:end]
Y = fac2[start:end]
plt.figure(figsize=(15,15))
plt.scatter(X, Y)
for i, x, y in zip(topMovies[start:end], X, Y):
    plt.text(x,y,movie_names[movies[i]], color=np.random.rand(3)*0.7, fontsize=14)
plt.show()

##  Neural net

Rather than creating a special purpose architecture (like our dot-product with bias earlier), it's often both easier and more accurate to use a standard neural network. Let's try it! Here, we simply concatenate the user and movie embeddings into a single vector, which we feed into the neural net.

In [ ]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [ ]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [ ]:
nn.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=8, 
          validation_data=([val.userId, val.movieId], val.rating))

This improves on our already impressive accuracy even further!